<div style="text-align: justify">
    <h1> Webscraping </h1> 
    
In this project we need to gather data to feed into our machine learning model in order to make predictions. This is where web scraping comes in. Web scraping is the process of extracting content and data from a website. There are various webscraping tools to choose from. We chose Selenium for our project because of its ability to interact with web elements and scrape sites with dynamic content. The data was gotten from a website known as forex factory, which focuses on monthly economic news releases of different economies. After extraction the data is inserted into a database so that it can be easily called up from our machine learning model.

 </div>

In [1]:
# Import all modules and load the database
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
import datetime
import re
import pandas as pd
%load_ext sql
%sql ibm_db_sa://

DB2/LINUXX8664


## Initializing variables and defining functions

In [2]:
expected_news = 'Unemployment Rate'
expected_currency = 'USD'

# Dictionary that maps the names of the news releases to their correspoonding table names in the database
table_dict = {'Federal Funds Rate': 'US_FOMC_RATES', 'ISM Manufacturing PMI': 'US_ISM_MANUFACTURING_PMI',
              'ISM Services PMI': 'US_ISM_SERVICES_PMI', 'CPI m/m': 'US_CPI', 'Retail Sales m/m': 'US_RETAIL_SALES',
              'Trade Balance': 'US_TRADE_BALANCE', 'Unemployment Rate': 'US_UNEMPLOYMENT_RATES', 
              'Non-Farm Employment Change': 'US_NON_FARM_EMPLOYMENT_CHANGE', 
              'Average Hourly Earnings m/m': 'US_AVERAGE_HOURLY_EARNINGS'}

# Dictionary that contains the news releases as keys and all the news releases that affects them in a list.
news_dict = {'Federal Funds Rate': ['ISM Manufacturing PMI', 'ISM Services PMI', 'CPI m/m', 'Retail Sales m/m', 'Trade Balance','Unemployment Rate', 'Non-Farm Employment Change', 'Average Hourly Earnings m/m'],
             'Unemployment Rate' : ['ISM Manufacturing PMI', 'ISM Services PMI', 'CPI m/m', 'Retail Sales m/m', 'Trade Balance', 'Federal Funds Rate']
            }

# Dictionary that maps the news to its future date table.
future_dates_dict = {'Federal Funds Rate': 'US_FOMC_FUTURE_DATES', 'Unemployment Rate': 'US_NFP_FUTURE_DATES'}

initial_date = ''
initial_time = ''

In [1]:
# Get the most recent date for the selected news release
query = 'SELECT MAX(DATE) FROM {}'.format(table_dict[expected_news])
previous_date = %sql $query
previous_date = previous_date[0][0]
previous_date

In [7]:
# This function checks if the date cell on the table is empty.
# If it is not empty the global variable initial_date is assigned the value 0f the date.
# If it is empty the date is assigned to the value of the initial_date.
def fill_empty_date(current):
    global initial_date
    
    if current:
        initial_date = current
    else:
        current = initial_date
    return current[0].text

# This function checks if the date cell on the table is empty.
# If it is not empty the global variable initial_date is assigned the value 0f the date.
# If it is empty the date is assigned to the value of the initial_date.
def fill_empty_time(current):
    global initial_time
    
    if current:
        initial_time = current
    else:
        current = initial_time
    return current

# This function converts a string into a date object.
def to_date(date):
    if len(date) <= 6:
        date = date + ' ' + year
    return datetime.datetime.strptime(date , '%b %d %Y').date()

# This function converts a string into a date object.
def to_time(time):
    try:
        return datetime.datetime.strptime(time, '%I:%M%p').time()
    except ValueError:
        return None
    
# This function removes all characters from a string except numbers, periods and minus signs. 
#Then it converts the string into a float.
# If there is a '%' sign it will divide the float by 100.
def to_float(value_as_string):
    value_as_float = float(re.sub('[^0-9.-]', '', value_as_string))
    if '%' in value_as_string:
        return value_as_float/100
    return value_as_float

## Opening the web page

In [ ]:
# Open the Forex Factory website with the month and year of the previous date
month = ['jan', 'feb', 'mar', 'apr', 'may', 'jun', 'jul', 'aug', 'sep', 'oct', 'nov', 'dec'][previous_date.month-1]
year = str(previous_date.year)

my_url = 'https://www.forexfactory.com/calendar?month=' + month + '.' + year

PATH = 'C:\Program Files (x86)\chromedriver.exe'
options = webdriver.ChromeOptions()
options.add_argument('--disable-blink-features=AutomationControlled')

driver = webdriver.Chrome(PATH, options=options)
driver.get(my_url)
driver.get(driver.current_url)

## Finding the next release date 

In [8]:
# Find the next release date of the news we are trying to predict 
table = driver.find_element_by_class_name('calendar__table')

for row in table.find_elements_by_css_selector('tr.calendar__row.calendar_row.calendar__row--grey'):

    date = row.find_elements_by_css_selector('td.calendar__cell.calendar__date.date span span')
    date = to_date(fill_empty_date(date))
    
    currency = row.find_element_by_css_selector('td.calendar__cell.calendar__currency.currency').text.strip()
     
    if currency.lower() != expected_currency.lower():
        continue

    title = row.find_element_by_css_selector('span.calendar__event-title').text
    
    if (title == expected_news) and (date == previous_date):
        detail = row.find_element_by_css_selector('td.calendar__cell.calendar__detail.detail')
        detail.click()
        print(title, date)
        break

details_tab = WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.CSS_SELECTOR, 'tr.calendar__row.calendar__details.details.calendar__details--detail.detail')))
next_release =  details_tab.find_element_by_css_selector('a.calendarspecs__nextdetails').text
print('Next release date: {}'.format(next_release))

Unemployment Rate 2021-06-04
Next release: Jul 2, 2021


In [9]:
# Set the website to show only news releases between the previous date and next release date
calendar_button = driver.find_element_by_css_selector('li.calendar__options.left')
webdriver.ActionChains(driver).move_to_element(calendar_button).click(calendar_button).perform()

begin_input = driver.find_element_by_name('flex[Calendar_mainCal][begindate]')
end_input = driver.find_element_by_name('flex[Calendar_mainCal][enddate]')
apply_button = driver.find_element_by_name('flexSettings')

begin_input.clear()
end_input.clear()
begin_input.send_keys(previous_date.strftime('%b %d, %Y'))
end_input.send_keys(next_release)

apply_button.click()

## Extracting the news releases

In [10]:
# Iterate through the table and find the news releases that are used to predict the current news and append them to a list
WebDriverWait(driver, 20).until(EC.staleness_of(table))
table = driver.find_element_by_class_name('calendar__table')
news_list = []

for row in table.find_elements_by_css_selector('tr.calendar__row.calendar_row.calendar__row--grey'):
    
    date = row.find_elements_by_css_selector('td.calendar__cell.calendar__date.date span span')
    date = to_date(fill_empty_date(date))
    
    time = row.find_elements_by_css_selector('td.calendar__cell.calendar__time.time')[0].text
    time = to_time(fill_empty_time(time))
    
    currency = row.find_element_by_css_selector('td.calendar__cell.calendar__currency.currency').text.strip()
     
    if currency.lower() != expected_currency.lower():
        continue
        
    title = row.find_element_by_css_selector('span.calendar__event-title').text
    
    if not any(n.lower() == title.lower() for n in news_dict.get(expected_news, [])):
        continue
    
    value = row.find_element_by_css_selector('td.calendar__cell.calendar__actual.actual').text
    
    news_list.append({'TITLE': title, 'DATE': date, 'TIME': time ,'VALUE': to_float(value)})
    
driver.close()

#### List of news releases

In [11]:
# The news releases that were scraped from the website with their corresponding date, time and value. 
pd.DataFrame(news_list)

,TITLE,DATE,TIME,VALUE
0,Trade Balance,2021-06-08,13:30:00,-68.9000
1,CPI m/m,2021-06-10,13:30:00,0.0060
2,Retail Sales m/m,2021-06-15,13:30:00,-0.0130
3,Federal Funds Rate,2021-06-16,19:00:00,0.0025


## Inserting data into database

In [2]:
# Add the scraped news releases into their corresponding tables in the database
# Making sure to check whether the scraped news release already exists within the database
for news in news_list:
    query = "SELECT DATE FROM {} WHERE DATE = '{}'".format(table_dict[news['TITLE']], news['DATE'])
    table_date = %sql $query
    if len(table_date) == 0:
        query = "INSERT INTO {} VALUES('{}', '{}', ({}))".format(table_dict[news['TITLE']], news['DATE'], news['TIME'], news['VALUE'])
        print(query)
        %sql $query

In [3]:
query = "DELETE FROM {}".format(future_dates_dict[expected_news])
%sql $query

In [4]:
query = "INSERT INTO {} VALUES('{}')".format(future_dates_dict[expected_news], to_date(next_release.replace(',', '')))
%sql $query